In [9]:
from datetime import datetime
from sqlalchemy import create_engine, Column, Integer, String, Float, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import csv

# Tworzenie bazy danych
engine = create_engine('sqlite:///weather_data.db')
Base = declarative_base()

# Definicja klasy Station
class Station(Base):
    __tablename__ = 'stations'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    country = Column(String)
    state = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

# Definicja klasy Measurement
class Measurement(Base):
    __tablename__ = 'measurements'
    id = Column(Integer, primary_key=True)
    station_id = Column(Integer)
    date = Column(Date)
    precip = Column(Float)
    tobs = Column(Float)

# Tworzenie tabel w bazie danych
Base.metadata.create_all(engine)

# Wczytywanie danych z plików CSV
with open('clean_stations.csv', 'r') as csvfile:
    station_data = list(csv.DictReader(csvfile))

with open('clean_measure.csv', 'r') as csvfile:
    measurement_data = list(csv.DictReader(csvfile))

# Tworzenie sesji
Session = sessionmaker(bind=engine)
session = Session()

# Wprowadzanie danych do tabeli stations
for data in station_data:
    station = Station(
        name=data['name'],
        country=data['country'],
        state=data['state'],
        latitude=float(data['latitude']),
        longitude=float(data['longitude']),
        elevation=float(data['elevation'])
    )
    session.add(station)

# Wprowadzanie danych do tabeli measurements
for data in measurement_data:
    measurement = Measurement(
        station_id=data['station'],
        date=datetime.strptime(data['date'], '%Y-%m-%d'),
        precip=float(data['precip']),
        tobs=float(data['tobs'])
    )
    session.add(measurement)

# Zapisywanie zmian w bazie danych
session.commit()

# Zapytanie do bazy danych
results = session.execute("SELECT * FROM stations LIMIT 5").fetchall()
for result in results:
    print(result)

# Zamykanie sesji
session.close()

(1, 'WAIKIKI 717.2', 'US', 'HI', 21.2716, -157.8168, 3.0)
(2, 'KANEOHE 838.1', 'US', 'HI', 21.4234, -157.8015, 14.6)
(3, 'KUALOA RANCH HEADQUARTERS 886.9', 'US', 'HI', 21.5213, -157.8374, 7.0)
(4, 'PEARL CITY', 'US', 'HI', 21.3934, -157.9751, 11.9)
(5, 'UPPER WAHIAWA 874.3', 'US', 'HI', 21.4992, -158.0111, 306.6)
